In [ ]:
#to unzip the file upload on the colab
from zipfile import ZipFile
file_name='/content/archive (1).zip'

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')
  

Done


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#pointer to data
train_dir = '/content/train'
val_dir = '/content/test'

#rescaling data so pixel lie between 0 to 1(to reduce useless data) 
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

#it add pointer to the data  and divide the data into batches(of size 64)
#we use batches to calculate the gradient decent and move in that direction(help train the more efficiently)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

#test data
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

import numpy as np
import cv2

#sequential means that our layer are added in linear fashion(input)
#A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor
from keras.models import Sequential
#
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D

emotion_model = Sequential()
#kernel is size of filter we apply on the data 
#we use relu function as mathematical function to change the out form linear to non linear model
#input shape is 3D 
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
#pooling layer help to remove excess information by applying a kernel=(3,3) to the image pixel 
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
#it dropout some randomly selected neuron (0.25)=25% of neuron are droppped out(it help to genearlise the model)
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# as input shape is 3D we require output in 1D(single value) so we need to flatten the layer
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

#compile-all layer of model are compile and ready to use
#metric=accuracy means that accuracy is the main parameter on which the answer(output) is decided
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

#now training the model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

emotion_model.save_weights('Emotion_recogniton_model.h5')





Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
448/448 [==============================] - 10s 22ms/step - loss: 1.7979 - accuracy: 0.2644 - val_loss: 1.6948 - val_accuracy: 0.3348
Epoch 2/50
448/448 [==============================] - 10s 22ms/step - loss: 1.6234 - accuracy: 0.3675 - val_loss: 1.5476 - val_accuracy: 0.4079
Epoch 3/50
448/448 [==============================] - 10s 21ms/step - loss: 1.5287 - accuracy: 0.4110 - val_loss: 1.4856 - val_accuracy: 0.4237
Epoch 4/50
448/448 [==============================] - 9s 21ms/step - loss: 1.4585 - accuracy: 0.4400 - val_loss: 1.4025 - val_accuracy: 0.4688
Epoch 5/50
448/448 [==============================] - 10s 22ms/step - loss: 1.3990 - accuracy: 0.4667 - val_loss: 1.3538 - val_accuracy: 0.4881
Epoch 6/50
448/448 [==============================] - 10s 21ms/step - loss: 1.3542 - accuracy: 0.4870 - val_loss: 1.3158

In [ ]:
from keras.models import load_model
nw=load_model('/content/Emotion_recogniton_model.h5')

In [ ]:
emotion_model.save('Emotion_recogniton.h5')

In [ ]:
from keras.models import load_model
nw=load_model('Emotion_recogniton.h5')

In [ ]:
/content/Emotion_recogniton.h5